![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/30.3.Text_Classification_with_FewShotClassifier.ipynb)

# Colab Setup

In [ ]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [2]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2  spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

# Installing tensorflow-addons
! pip install -q tensorflow-addons

In [3]:
import json
import os

import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.common import *
from sparknlp.training import *
from sparknlp.annotator import *

from sparknlp_jsl.base import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession

from sklearn.metrics import classification_report
import pandas as pd

params = {"spark.driver.memory":"16G",
          "spark.driver.maxResultSize":"2000M",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.serializer":"org.apache.spark.serializer.KryoSerializer"
          }

spark = sparknlp_jsl.start(secret = license_keys["SECRET"], params=params)

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 5.0.0
Spark NLP_JSL Version : 5.0.0


# Few Shot Classification

The `FewShotClassifierApproach` and `FewShotClassifierModel` annotators are new additions to the set of annotators available in the NLP library. These annotators specifically target few-shot classification tasks, which involve training a model to make accurate predictions with limited labeled data.

These new annotators provide a valuable capability for handling scenarios where labeled data is scarce or expensive to obtain. By effectively utilizing limited labeled examples, the few-shot classification approach enables the creation of models that can generalize and classify new instances accurately, even with minimal training data.

The FewShotClassifier is designed to process sentence embeddings as input. It generates category annotations, providing labels along with confidence scores that range from 0 to 1. Input annotation types supported by this model include `SENTENCE_EMBEDDINGS`, while the output annotation type is `CATEGORY`.

In [37]:
data = [
    ["ADE_positive", 'Both PAN and methotrexate have been independently demonstrated to cause sensorineural hearing loss.'],
    ["ADE_positive", 'Increased lash length, thickness, and pigmentation are well-documented side effects of prostaglandin analog glaucoma drops.'],
    ["ADE_positive", 'We reviewed the records of 3 patients with lymphoproliferative disorders who experienced acute coronary syndromes associated with their initial infusion of rituximab.'],
    ["ADE_positive", 'A 58-year-old woman with rheumatoid arthritis (RA) developed fever, skin eruptions, leukocytopenia, and thrombocytopenia, 3 weeks after treatment with sulfasalazine.'],
    ["ADE_positive", 'Adrenal suppression in a fetus due to administration of methylprednisolone has hitherto been rarely published.'],
    ["ADE_negative", 'Serum concentration of cerivastatin at 6 h after taking the last dose (0.15 mg) was 8062.5 ng/L, which was almost 5.7 times higher than that of normal persons.'],
    ["ADE_negative", 'The usual treatment includes quick relief bronchodilator medications of the sympathomimetic class and controller medications that may include the long-acting inhaled bronchodilator salmeterol.'],
    ["ADE_negative", 'Pathogenic mechanisms for the development of pseudomembranous colitis and the epidemiology of this condition in patients with AIDS are discussed.'],
    ["ADE_negative", 'On the basis of the clinico-radiologic presentation, a pulmonary hemorrhage was likely to occur; so to clarify the origin of this process, a complete serologic examination was performed but all the antibodies were negative.'],
    ["ADE_negative", 'I report a patient who developed the syndrome during treatment for schizophrenia with the antipsychotic agent molindone hydrochloride.']
]

In [38]:
train_data = spark.createDataFrame(data).toDF("label","text")
train_data.show(truncate=100)

+------------+----------------------------------------------------------------------------------------------------+
|       label|                                                                                                text|
+------------+----------------------------------------------------------------------------------------------------+
|ADE_positive| Both PAN and methotrexate have been independently demonstrated to cause sensorineural hearing loss.|
|ADE_positive|Increased lash length, thickness, and pigmentation are well-documented side effects of prostaglan...|
|ADE_positive|We reviewed the records of 3 patients with lymphoproliferative disorders who experienced acute co...|
|ADE_positive|A 58-year-old woman with rheumatoid arthritis (RA) developed fever, skin eruptions, leukocytopeni...|
|ADE_positive|Adrenal suppression in a fetus due to administration of methylprednisolone has hitherto been rare...|
|ADE_negative|Serum concentration of cerivastatin at 6 h after taking th

In [43]:
document_asm = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("sentence")

sentence_embeddings = BertSentenceEmbeddings\
.pretrained("sbiobert_base_cased_mli","en","clinical/models")\
    .setInputCols(["sentence"])\
    .setOutputCol("sentence_embeddings")

graph_builder = TFGraphBuilder()\
    .setModelName("fewshot_classifier")\
    .setInputCols(["sentence_embeddings"]) \
    .setLabelColumn("label")\
    .setGraphFolder("/tmp")\
    .setGraphFile("log_reg_graph.pb")\

few_shot_approach = FewShotClassifierApproach()\
    .setLabelColumn("label")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("prediction")\
    .setModelFile(f"/tmp/log_reg_graph.pb")\
    .setEpochsNumber(10)\
    .setBatchSize(1)\
    .setLearningRate(0.001)

pipeline = Pipeline(
    stages=[
        document_asm,
        sentence_embeddings,
        graph_builder,
        few_shot_approach
    ])

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]


In [44]:
%%time
model = pipeline.fit(train_data)

TF Graph Builder configuration:
Model name: fewshot_classifier
Graph folder: /tmp
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 768, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid'}
fewshot_classifier graph exported to /tmp/log_reg_graph.pb
CPU times: user 571 ms, sys: 41.8 ms, total: 612 ms
Wall time: 20.6 s


In [45]:
model.stages[-1].write().overwrite().save("/tmp/few_shot_model")

In [46]:
few_shot_model = sparknlp_jsl.annotators.FewShotClassifierModel.load("/tmp/few_shot_model")

## with LightPipeline

In [47]:
lpipeline = LightPipeline(Pipeline(
                stages=[
                    document_asm,
                    sentence_embeddings,
                    few_shot_model.setMultiClass(False)
                ]).fit(train_data))

In [51]:
tests = ['Bleomycin pneumonitis potentiated by oxygen administration.',
         'Enzymes derived from two different bacterial sources (Escherichia coli and Erwinia carotovora) are in common use.',
         'These cases and others quoted indicate that dependence on pressurized aerosol bronchodilators can and does occur in young people.',
         'PVT during chemotherapy in children is a rare event and appears to be closely related to intensive chemotherapy containing busulfan and to be associated with HVOD.',
         ]

In [52]:
for i, r in enumerate(lpipeline.fullAnnotate(tests)):
    print(i, r)


0 {'sentence': [Annotation(document, 0, 58, Bleomycin pneumonitis potentiated by oxygen administration., {}, [])], 'sentence_embeddings': [Annotation(sentence_embeddings, 0, 58, Bleomycin pneumonitis potentiated by oxygen administration., {'sentence': '0', 'token': 'Bleomycin pneumonitis potentiated by oxygen administration.', 'pieceId': '-1', 'isWordStart': 'true'}, [])], 'prediction': [Annotation(category, 0, 0, ADE_positive, {'confidence': '0.4356895'}, [])]}
1 {'sentence': [Annotation(document, 0, 112, Enzymes derived from two different bacterial sources (Escherichia coli and Erwinia carotovora) are in common use., {}, [])], 'sentence_embeddings': [Annotation(sentence_embeddings, 0, 112, Enzymes derived from two different bacterial sources (Escherichia coli and Erwinia carotovora) are in common use., {'sentence': '0', 'token': 'Enzymes derived from two different bacterial sources (Escherichia coli and Erwinia carotovora) are in common use.', 'pieceId': '-1', 'isWordStart': 'true'},

In [53]:
for i, r in enumerate(lpipeline.fullAnnotate(tests)):
    print(i ,r["prediction"][0].result)

0 ADE_positive
1 ADE_negative
2 ADE_negative
3 ADE_negative


In [55]:
for i, r in enumerate(lpipeline.fullAnnotate(tests)):
    print(i ,r["prediction"][0],r["sentence"][0].result )

0 Annotation(category, 0, 0, ADE_positive, {'confidence': '0.4356895'}, []) Bleomycin pneumonitis potentiated by oxygen administration.
1 Annotation(category, 0, 0, ADE_negative, {'confidence': '0.6639786'}, []) Enzymes derived from two different bacterial sources (Escherichia coli and Erwinia carotovora) are in common use.
2 Annotation(category, 0, 0, ADE_negative, {'confidence': '0.43239337'}, []) These cases and others quoted indicate that dependence on pressurized aerosol bronchodilators can and does occur in young people.
3 Annotation(category, 0, 0, ADE_negative, {'confidence': '0.6044256'}, []) PVT during chemotherapy in children is a rare event and appears to be closely related to intensive chemotherapy containing busulfan and to be associated with HVOD.


In [16]:
lpipeline.fullAnnotate('After 1 week of nefazodone therapy the patient experienced headache, confusion, and "gray areas" in her vision, without abnormal ophthalmologic findings.')

[{'sentence': [Annotation(document, 0, 152, After 1 week of nefazodone therapy the patient experienced headache, confusion, and "gray areas" in her vision, without abnormal ophthalmologic findings., {}, [])],
  'sentence_embeddings': [Annotation(sentence_embeddings, 0, 152, After 1 week of nefazodone therapy the patient experienced headache, confusion, and "gray areas" in her vision, without abnormal ophthalmologic findings., {'sentence': '0', 'token': 'After 1 week of nefazodone therapy the patient experienced headache, confusion, and "gray areas" in her vision, without abnormal ophthalmologic findings.', 'pieceId': '-1', 'isWordStart': 'true'}, [])],
  'prediction': [Annotation(category, 0, 0, ADE_positive, {'confidence': '0.68922853'}, [])]}]

In [17]:
lpipeline_multi = LightPipeline(Pipeline(
                      stages=[
                          document_asm,
                          sentence_embeddings,
                          few_shot_model.setMultiClass(True)
                      ]).fit(train_data))

In [18]:
for i, r in enumerate(lpipeline_multi.fullAnnotate(tests)):
    print(i,r)


0 {'sentence': [Annotation(document, 0, 58, Bleomycin pneumonitis potentiated by oxygen administration., {}, [])], 'sentence_embeddings': [Annotation(sentence_embeddings, 0, 58, Bleomycin pneumonitis potentiated by oxygen administration., {'sentence': '0', 'token': 'Bleomycin pneumonitis potentiated by oxygen administration.', 'pieceId': '-1', 'isWordStart': 'true'}, [])], 'prediction': [Annotation(category, 0, 0, ADE_negative, {'confidence': '0.581476'}, []), Annotation(category, 0, 0, ADE_positive, {'confidence': '0.57893157'}, [])]}
1 {'sentence': [Annotation(document, 0, 112, Enzymes derived from two different bacterial sources (Escherichia coli and Erwinia carotovora) are in common use., {}, [])], 'sentence_embeddings': [Annotation(sentence_embeddings, 0, 112, Enzymes derived from two different bacterial sources (Escherichia coli and Erwinia carotovora) are in common use., {'sentence': '0', 'token': 'Enzymes derived from two different bacterial sources (Escherichia coli and Erwini

In [19]:
for i, r in enumerate(lpipeline_multi.fullAnnotate(tests)):
    print(i, r["prediction"])


0 [Annotation(category, 0, 0, ADE_negative, {'confidence': '0.581476'}, []), Annotation(category, 0, 0, ADE_positive, {'confidence': '0.57893157'}, [])]
1 [Annotation(category, 0, 0, ADE_negative, {'confidence': '0.63793814'}, []), Annotation(category, 0, 0, ADE_positive, {'confidence': '0.5811099'}, [])]
2 [Annotation(category, 0, 0, ADE_negative, {'confidence': '0.5904401'}, []), Annotation(category, 0, 0, ADE_positive, {'confidence': '0.29843384'}, [])]
3 [Annotation(category, 0, 0, ADE_negative, {'confidence': '0.55433303'}, []), Annotation(category, 0, 0, ADE_positive, {'confidence': '0.39507768'}, [])]


In [20]:
lpipeline_multi.fullAnnotate('After 1 week of nefazodone therapy the patient experienced headache, confusion, and "gray areas" in her vision, without abnormal ophthalmologic findings.')

[{'sentence': [Annotation(document, 0, 152, After 1 week of nefazodone therapy the patient experienced headache, confusion, and "gray areas" in her vision, without abnormal ophthalmologic findings., {}, [])],
  'sentence_embeddings': [Annotation(sentence_embeddings, 0, 152, After 1 week of nefazodone therapy the patient experienced headache, confusion, and "gray areas" in her vision, without abnormal ophthalmologic findings., {'sentence': '0', 'token': 'After 1 week of nefazodone therapy the patient experienced headache, confusion, and "gray areas" in her vision, without abnormal ophthalmologic findings.', 'pieceId': '-1', 'isWordStart': 'true'}, [])],
  'prediction': [Annotation(category, 0, 0, ADE_negative, {'confidence': '0.35451746'}, []),
   Annotation(category, 0, 0, ADE_positive, {'confidence': '0.68922853'}, [])]}]

## with transform

In [21]:
test_data = spark.createDataFrame([
    ['After 1 week of nefazodone therapy the patient experienced headache, confusion, and "gray areas" in her vision, without abnormal ophthalmologic findings.'],
    ['Possible linkage of amprenavir with intracranial bleeding in an HIV-infected hemophiliac.'],
    ['Results Radiographic responses to sirolimus were observed in all patients.'],
    ['Obtaining appropriate cultures can be critical in making the diagnosis and directing treatment.']
]).toDF("text")

In [22]:
result = model.transform(test_data)

result.select("text","prediction").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------+
|text                                                                                                                                                     |prediction                                                      |
+---------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------+
|After 1 week of nefazodone therapy the patient experienced headache, confusion, and "gray areas" in her vision, without abnormal ophthalmologic findings.|[{category, 0, 0, ADE_positive, {confidence -> 0.6892285}, []}] |
|Possible linkage of amprenavir with intracranial bleeding in an HIV-infected hemophiliac.                          

In [23]:
model_multi = Pipeline(
                  stages=[
                      document_asm,
                      sentence_embeddings,
                      few_shot_model.setMultiClass(True)
                  ]).fit(train_data)

result_2 = model_multi.transform(test_data)

result_2.select("text","prediction").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                     |prediction                                                                                                                      |
+---------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------+
|After 1 week of nefazodone therapy the patient experienced headache, confusion, and "gray areas" in her vision, without abnormal ophthalmologic 

## with word embeddins

In [26]:
tokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
    .setInputCols(["sentence","token"])\
    .setOutputCol("word_embeddings")

sentence_embeddings = SentenceEmbeddings() \
    .setInputCols(["sentence", "word_embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

pipeline = Pipeline(
    stages=[
        document_asm,
        tokenizer,
        word_embeddings,
        sentence_embeddings,
        few_shot_approach
    ])

model = pipeline.fit(spark.createDataFrame([["",""]]).toDF("label","text"))

lpipeline = LightPipeline(model)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]


In [27]:
lpipeline.fullAnnotate('Nephrotic syndrome associated with interferon-beta-1b therapy for multiple sclerosis.')

[{'word_embeddings': [Annotation(word_embeddings, 0, 8, Nephrotic, {'isOOV': 'false', 'pieceId': '-1', 'isWordStart': 'true', 'token': 'Nephrotic', 'sentence': '0'}, []),
   Annotation(word_embeddings, 10, 17, syndrome, {'isOOV': 'false', 'pieceId': '-1', 'isWordStart': 'true', 'token': 'syndrome', 'sentence': '0'}, []),
   Annotation(word_embeddings, 19, 28, associated, {'isOOV': 'false', 'pieceId': '-1', 'isWordStart': 'true', 'token': 'associated', 'sentence': '0'}, []),
   Annotation(word_embeddings, 30, 33, with, {'isOOV': 'false', 'pieceId': '-1', 'isWordStart': 'true', 'token': 'with', 'sentence': '0'}, []),
   Annotation(word_embeddings, 35, 52, interferon-beta-1b, {'isOOV': 'false', 'pieceId': '-1', 'isWordStart': 'true', 'token': 'interferon-beta-1b', 'sentence': '0'}, []),
   Annotation(word_embeddings, 54, 60, therapy, {'isOOV': 'false', 'pieceId': '-1', 'isWordStart': 'true', 'token': 'therapy', 'sentence': '0'}, []),
   Annotation(word_embeddings, 62, 64, for, {'isOOV': '

# MTSamples Dataset

In [4]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/mtsamples_classifier.csv

In [5]:
spark_df = spark.read.csv("mtsamples_classifier.csv", header = True)

spark_df.show(10,truncate=100)

+----------------+----------------------------------------------------------------------------------------------------+
|        category|                                                                                                text|
+----------------+----------------------------------------------------------------------------------------------------+
|Gastroenterology| PROCEDURES PERFORMED: Colonoscopy. INDICATIONS: Renewed symptoms likely consistent with active f...|
|Gastroenterology| OPERATION 1. Ivor-Lewis esophagogastrectomy. 2. Feeding jejunostomy. 3. Placement of two right-s...|
|Gastroenterology| PREOPERATIVE DIAGNOSES: 1. Gastroesophageal reflux disease. 2. Chronic dyspepsia. POSTOPERATIVE ...|
|Gastroenterology| PROCEDURE: Colonoscopy. PREOPERATIVE DIAGNOSES: Rectal bleeding and perirectal abscess. POSTOPER...|
|Gastroenterology| PREOPERATIVE DIAGNOSIS: Right colon tumor. POSTOPERATIVE DIAGNOSES: 1. Right colon cancer. 2. As...|
|Gastroenterology| PREOPERATIVE DIAGNOSI

In [6]:
spark_df.groupBy("category").count().show()

+----------------+-----+
|        category|count|
+----------------+-----+
|         Urology|  115|
|       Neurology|  143|
|      Orthopedic|  223|
|Gastroenterology|  157|
+----------------+-----+



In [10]:
(trainingData, testData) = spark_df.randomSplit([0.8, 0.2], seed = 42)
(trainingData_part1, trainingData_part2) = trainingData.randomSplit([0.5, 0.5], seed = 42)

print("trainingData_with_embeddings:       ",trainingData.count())
print("testData_with_embeddings:           ",testData.count())
print("trainingData_part1_with_embeddings: ",trainingData_part1.count())

trainingData_with_embeddings:        536
testData_with_embeddings:            102
trainingData_part1_with_embeddings:  280


We will extract [sbiobert_base_cased_mli](https://nlp.johnsnowlabs.com/2020/11/27/sbiobert_base_cased_mli_en.html) embeddings which has 768 dimension output and use this embeddings in the model training.

In [11]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

bert_sent = BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", 'en','clinical/models')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

embeddings_pipeline = Pipeline(
    stages = [document_assembler,
              bert_sent])

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]


In [12]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")

trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMISSION DIAGNOSIS: Symptomatic cholelithiasis. DISCHAR...|Gastroenterology|[{sentence_embeddings, 0, 2228,  ADMISSION DIAGNOSIS: Sym...|
| ADMITTING DIAGNOSES: Hiatal hernia, gastroesophageal ref...|Gastroenterology|[{sentence_embeddings, 0, 3237,  ADMITTING DIAGNOSES: Hia...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows



In [13]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")

testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMITTING DIAGNOSIS: Gastrointestinal bleed. HISTORY OF ...|Gastroenterology|[{sentence_embeddings, 0, 3978,  ADMITTING DIAGNOSIS: Gas...|
| CHIEF COMPLAINT: Dysphagia and hematemesis while vomitin...|Gastroenterology|[{sentence_embeddings, 0, 6515,  CHIEF COMPLAINT: Dysphag...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows



In [14]:
trainingData_part1_with_embeddings = embeddings_pipeline.fit(trainingData_part1).transform(trainingData_part1)\
                                                  .select("text","category","sentence_embeddings")

trainingData_part2_with_embeddings = embeddings_pipeline.fit(trainingData_part2).transform(trainingData_part2)\
                                                  .select("text","category","sentence_embeddings")

### ClassifierDL

In [15]:
log_folder="classifier_dl_logs_bert"

In [16]:
classifier_dl = ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("prediction_class")\
        .setLabelColumn("category")\
        .setBatchSize(8)\
        .setMaxEpochs(10)\
        .setLr(0.002)\
        .setDropout(0.1)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = Pipeline(stages=[classifier_dl])

In [17]:
%%time
clfDL_model_bert = classifier_dl_pipeline.fit(trainingData_with_embeddings)

CPU times: user 3.85 s, sys: 493 ms, total: 4.34 s
Wall time: 10min 57s


In [23]:
preds = clfDL_model_bert.transform(testData_with_embeddings)

preds_df = preds.select("category","text","prediction_class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print(classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.88      0.88      0.88        25
       Neurology       0.75      0.82      0.78        22
      Orthopedic       0.85      0.80      0.82        35
         Urology       0.90      0.90      0.90        20

        accuracy                           0.84       102
       macro avg       0.84      0.85      0.85       102
    weighted avg       0.85      0.84      0.84       102



In [24]:
results_df = pd.DataFrame(columns=["macro-f1-score","weighted-f1-score","accuracy"])

res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["ClassifierDL_full_Data"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

In [25]:
results_df

,macro-f1-score,weighted-f1-score,accuracy
ClassifierDL_full_Data,0.846535,0.843538,0.843137


### ClassifierDL partial

In [26]:
log_folder="classifier_dl_logs_bert"

In [27]:
classifier_dl = ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("prediction_class")\
        .setLabelColumn("category")\
        .setBatchSize(8)\
        .setMaxEpochs(10)\
        .setLr(0.002)\
        .setDropout(0.1)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = Pipeline(stages=[classifier_dl])

In [28]:
%%time

clfDL_model_bert = classifier_dl_pipeline.fit(trainingData_part1_with_embeddings)

CPU times: user 1.94 s, sys: 214 ms, total: 2.15 s
Wall time: 5min 44s


In [29]:
preds = clfDL_model_bert.transform(testData_with_embeddings)

preds_df = preds.select("category","text","prediction_class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.81      0.88      0.85        25
       Neurology       0.79      0.86      0.83        22
      Orthopedic       0.88      0.86      0.87        35
         Urology       0.88      0.75      0.81        20

        accuracy                           0.84       102
       macro avg       0.84      0.84      0.84       102
    weighted avg       0.85      0.84      0.84       102



In [30]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["ClassifierDL_partial_Data"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

In [31]:
results_df

,macro-f1-score,weighted-f1-score,accuracy
ClassifierDL_full_Data,0.846535,0.843538,0.843137
ClassifierDL_partial_Data,0.838154,0.842929,0.843137


## Few Shot partial

In [43]:
graph_builder = TFGraphBuilder()\
    .setModelName("fewshot_classifier")\
    .setInputCols(["sentence_embeddings"]) \
    .setLabelColumn("category")\
    .setGraphFolder("/tmp")\
    .setGraphFile("log_reg_graph.pb")\

few_shot_approach = FewShotClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("prediction")\
    .setModelFile(f"/tmp/log_reg_graph.pb")\
    .setEpochsNumber(10)\
    .setBatchSize(1)\
    .setLearningRate(0.001)

pipeline = Pipeline(
    stages=[
        graph_builder,
        few_shot_approach
    ])

In [44]:
%%time
model = pipeline.fit(trainingData_part1_with_embeddings)

TF Graph Builder configuration:
Model name: fewshot_classifier
Graph folder: /tmp/
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 768, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid'}
fewshot_classifier graph exported to /tmp//log_reg_graph.pb
CPU times: user 3.7 s, sys: 391 ms, total: 4.09 s
Wall time: 9min 40s


In [45]:
preds = model.transform(testData_with_embeddings)

In [36]:
from sklearn.metrics import  classification_report

preds_df = preds.select("category","text","prediction.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.88      0.92      0.90        25
       Neurology       0.81      0.77      0.79        22
      Orthopedic       0.91      0.89      0.90        35
         Urology       0.86      0.90      0.88        20

        accuracy                           0.87       102
       macro avg       0.87      0.87      0.87       102
    weighted avg       0.87      0.87      0.87       102



In [37]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["FewShot_partial_Data"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

In [38]:
results_df

,macro-f1-score,weighted-f1-score,accuracy
ClassifierDL_full_Data,0.846535,0.843538,0.843137
ClassifierDL_partial_Data,0.838154,0.842929,0.843137
FewShot_partial_Data,0.867314,0.872104,0.872549
